In [3]:
from chromalab.adaptation import dead_leaves

import matplotlib.colors as mcolors
import numpy as np
import polyscope as ps  # https://polyscope.run/py/
import random

from pathlib import Path

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

In [3]:
# Initialize polyscope and set some scene options
ps.init()

ps.set_build_gui(False)
ps.set_ground_plane_mode("none")
ps.set_SSAA_factor(4)
ps.set_background_color([0, 0, 0])

image_res = 512
ps.set_window_resizable(True)
ps.set_window_size(image_res / 2, image_res / 2)

[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 4.1 INTEL-22.5.11


In [4]:
# Set up directories for saving outputs
Path('./out/tri').mkdir(parents=True, exist_ok=True)
Path('./out/RG1G2B').mkdir(parents=True, exist_ok=True)
Path('./out/LMS_Q').mkdir(parents=True, exist_ok=True)
Path('./out/four_gray').mkdir(parents=True, exist_ok=True)

In [5]:
n_channels = 4

if n_channels == 3:
    # Generate some RGB colors
    hues = np.linspace(0, 0.9, 100)
    color_palette = np.array([mcolors.hsv_to_rgb([hue, 1, 1]) for hue in hues])

    # Set up the dead leaves scene
    dead_leaves(0.5, color_palette, max_iters=20, shape_mode='mixed')

    # Take a screenshot
    dir = './out/tri/'
    ps.screenshot(dir + 'dead_leaves_tri.png')

    # Write a video file
    fd = ps.open_video_file(dir + 'dead_leaves_tri.mp4', fps=60)
    for i in range(60):
        # Add in some camera motion here per frame
        ps.write_video_frame(fd)
    ps.close_video_file(fd)

ffmpeg version 7.0.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --ena

In [ ]:
if n_channels == 4:
    # Generate some tetra colors
    hues = np.linspace(0, 0.9, 100)
    color_palette = np.array([mcolors.hsv_to_rgb([hue, 1, 1]) for hue in hues])
    Qs = np.random.rand(color_palette.shape[0], 1)
    color_palette = np.hstack([color_palette, Qs])

    # Set up the dead leaves scene
    dead_leaves(0.5, color_palette, max_iters=20, shape_mode='mixed')
    
    """
    save_image_mode -- if rasterizing a tetracolor scene, mode to save the output (to either image or video file)
        'LMS_Q' : saves first 3 channels in one RGB .png file, saves 4th channel in one grayscale .png file
        'four_gray' : saves each channel to a separate grayscale .png file
        'RG1G2B' : saves 4 color channels in one RGBA .png file
    """
    tetra_save_mode = 'LMS_Q'
    # tetra_save_mode = 'four_gray'
    # tetra_save_mode = 'RG1G2B'

    dir = f'./out/{tetra_save_mode}/'

    # Take a tetra screenshot
    ps.rasterize_tetra(dir + 'dead_leaves_tetra.png', save_image_mode=tetra_save_mode)

    # Write a tetra video file
    # 'RG1G2B' is a valid save_image_mode for writing video files, but mp4 won't write transparency
    # Most likely want to use 'LMS_Q' or 'four_gray' mode for writing tetra video files!
    tfds = ps.open_tetra_video_file(dir + 'dead_leaves_tetra.mp4', fps=30, save_image_mode=tetra_save_mode)
    for i in range(30):
        # Add in some camera motion
        ps.write_tetra_video_frame(tfds)
    ps.close_tetra_video_file(tfds)